In [1]:
import VariantCalling as vc
import numpy as np

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers.legacy import Adam

In [3]:
dg = vc.VariantCallingData()

In [4]:
# # Check if GPU is available
# if tf.config.experimental.list_physical_devices('GPU'):
#     # Limit GPU memory allocation to avoid GPU memory exhaustion
#     for gpu in tf.config.experimental.list_physical_devices('GPU'):
#         tf.config.experimental.set_memory_growth(gpu, True)
# else:
#     print("No GPU found. Using CPU.")


In [5]:
def create_cnn_model1(input_shape):
    input_layer = Input(shape=input_shape)

    # Encoder
    encoded = Conv2D(64, (8, 8), activation='relu', padding='same')(input_layer)
    encoded = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)

    # Decoder
    decoded = Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(encoded)
    decoded = Conv2DTranspose(64, (8, 8), activation='relu', padding='same')(decoded)
    decoded = Conv2D(1, (8, 8), activation='sigmoid', padding='same')(decoded)

    model = Model(input_layer, decoded)

    return model

def create_cnn_model2(input_shape):
    input_layer = Input(shape=input_shape)

    # Encoder
    encoded = Conv2D(64, (5, 5), activation='relu', padding='same')(input_layer)
    encoded = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)

    # Decoder
    decoded = Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(encoded)
    decoded = Conv2DTranspose(64, (5, 5), activation='relu', padding='same')(decoded)
    decoded = Conv2D(1, (5, 5), activation='sigmoid', padding='same')(decoded)

    model = Model(input_layer, decoded)

    return model

In [6]:
def gen_data(align_err, seq_err):
    img_row = 100
    img_col = 178
    
    print(f"\ngenerating alignment error {align_err}, seq error {seq_err}")
    noisy, clean = dg.generate_data_for_noise_reduction(sample_size=15000, alignment_error_prob=align_err, sequencing_error_prob=seq_err)
    
    noisy = noisy.astype('float32') / 3
    clean = clean.astype('float32') / 3
    noisy = noisy.reshape(noisy.shape[0], img_row, img_col, 1)
    clean = clean.reshape(clean.shape[0], img_row, img_col, 1)
    return noisy, clean

In [7]:
input_shape = (100, 178, 1)

model1 = create_cnn_model1(input_shape)
model2 = create_cnn_model2(input_shape)

learning_rate = 0.0005
model1.compile(optimizer=Adam(learning_rate=learning_rate), loss=MeanSquaredError())
print("\nmodel1:")
print(model1.summary())

learning_rate = 0.0005
model2.compile(optimizer=Adam(learning_rate=learning_rate), loss=MeanSquaredError())
print("\nmodel2:")
print(model2.summary())


2023-09-02 02:56:19.349237: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-09-02 02:56:19.349262: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-09-02 02:56:19.349267: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-09-02 02:56:19.349297: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-02 02:56:19.349314: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)



model1:
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 178, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 100, 178, 64)      4160      
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 178, 128)     73856     
                                                                 
 conv2d_transpose (Conv2DTr  (None, 100, 178, 128)     147584    
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 100, 178, 64)      524352    
 Transpose)                                                      
                                                                 
 conv2d_2 (Conv2D)           (None, 100, 178, 1)    

In [8]:
# Curriculum
batch_size = 32
img_row = 100
img_col = 178

curriculum = [
    {"align_err": 0, "seq_err": 0.01, "model1_epochs": 8, "model2_epochs": 4},
    {"align_err": 0, "seq_err": 0.05, "model1_epochs": 5, "model2_epochs": 3},
    {"align_err": 0.001, "seq_err": 0.05, "model1_epochs": 7, "model2_epochs": 3},
    {"align_err": 0.003, "seq_err": 0.05, "model1_epochs": 5, "model2_epochs": 3},
    {"align_err": 0.005, "seq_err": 0.05, "model1_epochs": 5, "model2_epochs": 2},
    {"align_err": 0.01, "seq_err": 0.05, "model1_epochs": 5, "model2_epochs": 2},
]

for module in curriculum:
    print("current module: ", module)
    noisy, clean = gen_data(module["align_err"], module["seq_err"])

    epochs = module["model1_epochs"]
    model1.fit(noisy, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

    denoised_image1 = model1.predict(noisy)
    denoised_image1_quantised = np.round(denoised_image1 * 3) / 3

    epochs = module["model2_epochs"]
    model2.fit(denoised_image1_quantised, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

current module:  {'align_err': 0, 'seq_err': 0.01, 'model1_epochs': 8, 'model2_epochs': 4}

generating alignment error 0, seq error 0.01


  0%|          | 0/15000 [00:00<?, ?it/s]

Epoch 1/8


2023-09-02 03:00:20.042275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 382s 814ms/step - loss: 0.0047
Epoch 2/8
175/469 [==========>...................] - ETA: 3:57 - loss: 5.3220e-04

KeyboardInterrupt: 